<a href="https://colab.research.google.com/github/mgamzec/GeospatialData-Python/blob/main/geospatial_machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geospatial machine learning

In this notebook, we will introduce the field of geospatial machine learning by first going over the geospatial data primitives then solving a machine learning problem in an "end-to-end" fashion.

We aim to cover the following:
1. **Introduction to geospatial data**: vector and raster data primitives.
2. **Problem framing**: introducing the problem that we are going to solve.
3. **Data** acquisition and preprocessing: we will get the data and preprocess it for machine learning.
4. **Model fitting**: we will fit a model to the data and conduct hyperparameter search.
5. **Model evaluation**: we will evaluate the model on the test data.
6. **Inference**: we will predict the output for the test data.

Before we start, let's install the necessary libraries:

In [ ]:
%pip install rioxarray -q
%pip install shap -q
%pip install contextily -q

Next, we need to download the necessary files to be used in this practical:

In [ ]:
from pathlib import Path
import requests
import shutil
import os

# Set the URLs of the files
raster_url = "https://drive.google.com/uc?export=download&id=1CrizA11Ri3jBtlMLu-58MDkM_aELt9RQ"
crop_url = "https://drive.google.com/uc?export=download&id=1w1pvR0ESImXhgoCdy3QO6dV03vXbmvHH"
bikes_url = "https://drive.google.com/uc?export=download&id=161vAJpEnau9pXuJEi0omDmNu38cudJq1"
paris_districts_url = "https://drive.google.com/uc?export=download&id=1XyM6U-rO963zRDmtHAUx918De3qmMqzz"

def download_file(url, directory, file_name):

    print(f"Saving `{file_name}` to `{directory}/`")

    # Create directory if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)

    file_path = os.path.join(directory, file_name)

    # Send a HTTP request to the URL of the file
    response = requests.get(url, stream=True)

    # Throw an error for bad status codes
    response.raise_for_status()

    # Write the file
    with open(file_path, 'wb') as handle:
        for block in response.iter_content(1024):
            handle.write(block)

# Set the path of the directory
data_dir = Path("./files")
if data_dir.exists(): shutil.rmtree(data_dir, ignore_errors=True)
data_dir.mkdir(exist_ok=True)

# Download
download_file(raster_url, data_dir, "elevation.tiff")
download_file(crop_url, data_dir, "df.feather")
download_file(bikes_url, data_dir, "paris_bike_stations_mercator.gpkg")
download_file(paris_districts_url, data_dir, "paris_districts_utm.geojson")

---

# Introduction to Geospatial Data

Geospatial data refers to **information that can be associated with geographic locations on Earth**. It comes with spatial attributes such as location and geometry shape.

Examples of geospatial data:
- Population density.
- Weather and climate information.
- Transportation networks.

Typically, geospatial data is represented in two ways:

<div style="text-align:center;"> <figure> <img style="width:33%;" src="https://i0.wp.com/pangeography.com/wp-content/uploads/2022/05/Raster_vector_tikz.png" /> <figcaption style="font-size:small;">Image credit: <a href="https://pangeography.com/geographic-data-structure-vector-data-and-raster-data/">Pan Geography</a></figcaption> </figure> </div>

- **Vector**: points, lines, polygons, etc. Each vector object is a geometry that can have multiple attributes. Such data is typically saved as a vector file (`Shapefile` (.shp) and `GeoJSON`, among many).
- **Raster**: similar to images, it is represented as a grid of cells or pixels, each cell holds a value representing a value or measurement. Raster data is typically stored in formats such as `GeoTIFF` or `NetCDF`.

Next, let's get an overview of **vector** and **raster** data.

## Vector

- You'd want to use `shapely` to create and manipulate geometry objects in Python.
- `shapely` provides support for gegraphic information systems operations, such as **spatial relationships**, **geometric operations**, and **coordinate transformations**.

... but in most cases, using the higher-level library **geopandas** would be enough.

### Basic Geometric Types

`shapely` allows us to create basic geometric objects that are commonly used in geospatial analysis. Examples:
- `Point`: Represents a single point in 2-3 D space.
- `LineString`: Represents a sequence of connected points forming a line.
- `Polygon`: Represents a filled area defined by a sequence of points that form a closed ring.

Let's start by creating a `point`:

In [ ]:
from shapely.geometry import Point

# Create a point
point = Point(1,2)
point

... how about a polygon (enclosed list of coordinates)?

In [ ]:
from shapely.geometry import Polygon

points = [(0,0), (0,2), (2,2), (2,0)]
polygon = Polygon(points)
polygon

We can also create multiple geometries:

In [ ]:
from shapely.geometry import MultiPoint, MultiLineString, MultiPolygon, LineString

points = [Point(0, 0), Point(1, 1), Point(2, 2)]
multipoint = MultiPoint(points)
multipoint

In [ ]:
linestrings = [LineString([(0, 0), (1, 1)]), LineString([(1, 1), (0, 2)])]
multiline = MultiLineString(linestrings)
multiline

In [ ]:
polygons = [Polygon([(0, 0), (0, 1), (1, 1), (1, 0), (0, 0)]), Polygon([(1, 1), (1, 2), (2, 2), (2, 1), (1, 1)])]
multipolygon = MultiPolygon(polygons)
multipolygon

### Properties of Geometric Objects

After we create geometric objects, we can access attributes that can be useful in pratice. We highlight a few important ones:

- `area`: Returns the area of a `Polygon` or `MultiPolygon` object.
- `bounds`: Returns the bounding box of a geometric object as a tuple `(min_x, min_y, max_x, max_y)`.
- `centroid`: Returns the geometric centroid of a geometric object as a `Point`.

In [ ]:
# Area of a Polygon
polygon_area = polygon.area
polygon_area

In [ ]:
# Bounds of a Point
point_bounds = point.bounds
print(f"Point bounds: {point_bounds}")
print(f"Point coordinates: {point.x, point.y}")

In [ ]:
# Centroid of a Polygon
polygon_centroid = polygon.centroid
polygon_centroid

On top of attributes, we have spatial operations and relationships that can take multiple geometries and produce new ones. We highlight the following:
- `union`: Computes the geometric union of two objects.
- `intersection`: Computes the geometric intersection of two objects.
- `difference`: Computes the geometric difference of two objects.
- `contains`: whether one geometric object contains another.
- `intersects`: whether two geometric objects intersect.

In [ ]:
# Create two polygon objects
polygon1 = Polygon([(0, 0), (0, 2), (2, 2), (2, 0), (0, 0)])
polygon2 = Polygon([(1, 1), (1, 3), (3, 3), (3, 1), (1, 1)])

# Union of two polygons
polygon_union = polygon1.union(polygon2)
polygon_union

In [ ]:
# Intersection of two polygons
polygon_intersection = polygon1.intersection(polygon2)
polygon_intersection

In [ ]:
# The difference
polygon1.difference(polygon2)

In [ ]:
# A bunch of points
point1 = Point(0, 0)
point2 = Point(0, 0)
point3 = Point(1, 1)

# One linestring (connected points)
linestring = LineString([(0, 0), (1, 1), (2, 2)])

# Two polygons
polygon1 = Polygon([(0, 0), (0, 2), (2, 2), (2, 0), (0, 0)])
polygon2 = Polygon([(1, 1), (1, 3), (3, 3), (3, 1), (1, 1)])

# Spatial relationship predicates
print("point1 equals point2:", point1.equals(point2))
print("point1 within polygon1:", point1.within(polygon1))
print("polygon1 intersects polygon2:", polygon1.intersects(polygon2))
print("polygon1 touches polygon2:", polygon1.touches(polygon2))

In most cases, however, we'll be using `geopandas` to read vector files and analyze the data:

In [ ]:
import geopandas as gpd

# Load the countries dataframe using geopandas
countries = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

We can use methods like `head()`, `info()`, and `describe()` to inspect and explore GeoDataFrames, similar to how you would use them with Pandas DataFrames:

In [ ]:
countries.head()

In [ ]:
countries.info()

Cooordinate reference systems (CRS) can take you from the geometric coordinates (numbers) to the earth's surface. `GeoPandas` allows us to inspect the CRS and reproject it if necessary:

In [ ]:
# Check the CRS
countries.crs

We can use `shapely` attributes and operations to get geometries of interest. Example:

In [ ]:
# Plot the union of all african countries
countries[countries["continent"] == "Africa"].unary_union

### Exercises

<div class="alert alert-success">

**EXERCISE**:

We will start with exploring the bicycle station dataset (available as a GeoPackage file: `data/paris_bike_stations_mercator.gpkg`)
    
* Read the stations datasets into a GeoDataFrame called `stations`.
* Check the type of the returned object
* Check the first rows of the dataframes. What kind of geometries does this datasets contain?
* How many features are there in the dataset?
    
<details><summary>Hints</summary>

* Use `type(..)` to check any Python object type
* The `geopandas.read_file()` function can read different geospatial file formats. You pass the file name as first argument.
* Use the `.shape` attribute to get the number of features

</details>
    
    
</div>

<div class="alert alert-success">

**EXERCISE**:

* Make a quick plot of the `stations` dataset.
* Make the plot a bit larger by setting the figure size to (12, 6) (hint: the `plot` method accepts a `figsize` keyword).

</div>

A plot with points can be hard to interpret without any spatial context. Therefore, we will learn how to add a background map.

We are going to make use of the [contextily](https://github.com/darribas/contextily) package. the `add_basemap()` function of this package makes it easy to add a background web map to our plot. We begin by plotting our data, then pass the matplotlib axes object to the `add_basemap()` function. `contextily` will then download the web tiles needed for the geographical extent of the plot.

<div class="alert alert-success">

**EXERCISE**:

* Import `contextily`.
* Re-do the figure of the previous exercise: make a plot of all the points in `stations`, but assign the result to an `ax` variable.
* Set the marker size equal to 5 to reduce the size of the points (use the `markersize` keyword of the `plot()` method for this).
* Use the `add_basemap()` function of `contextily` to add a background map: the first argument is the matplotlib axes object `ax`.

</div>

<div class="alert alert-success">

**EXERCISE**:

* Make a histogram showing the distribution of the number of bike stands in the stations.

<details>
  <summary>Hints</summary>

* Selecting a column can be done with the square brackets: `df['col_name']`
* Single columns have a `hist()` method to plot a histogram of its values.
    
</details>
    
</div>

<div class="alert alert-success">

**EXERCISE**:

Let's now visualize where the available bikes are actually stationed:
    
* Make a plot of the `stations` dataset (also with a (12, 6) figsize).
* Use the `'available_bikes'` columns to determine the color of the points. For this, use the `column=` keyword.
* Use the `legend=True` keyword to show a color bar.

</div>

<div class="alert alert-success">

**EXERCISE**:

Next, we will explore the dataset on the administrative districts of Paris (available as a GeoJSON file: "data/paris_districts_utm.geojson")

* Read the dataset into a GeoDataFrame called `districts`.
* Check the first rows of the dataframe. What kind of geometries does this dataset contain?
* How many features are there in the dataset? (hint: use the `.shape` attribute)
* Make a quick plot of the `districts` dataset (set the figure size to (12, 6)).
    
</div>

<div class="alert alert-success">

**EXERCISE**:
    
What are the largest districts (biggest area)?

* Calculate the area of each district.
* Add this area as a new column to the `districts` dataframe.
* Sort the dataframe by this area column for largest to smallest values (descending).

<details><summary>Hints</summary>

* Adding a column can be done by assigning values to a column using the same square brackets syntax: `df['new_col'] = values`
* To sort the rows of a DataFrame, use the `sort_values()` method, specifying the colum to sort on with the `by='col_name'` keyword. Check the help of this method to see how to sort ascending or descending.

</details>

</div>

<div class="alert alert-success">

**EXERCISE**:

* Add a column `'population_density'` representing the number of inhabitants per squared kilometer (Note: The area is given in squared meter, so you will need to multiply the result with `10**6`).
* Plot the districts using the `'population_density'` to color the polygons. For this, use the `column=` keyword.
* Use the `legend=True` keyword to show a color bar.

</div>

## Raster

To read raster data, we need a library capable of reading geo file formats (e.g. GeoTIFF, NetCDF, etc.) and preserving their metadata.

A library that can do that and is integrated with the Python ecosystem (NumPy, Geopandas, Rasterio, etc.) is `rioxarray`. `rioxarray` supports many critical tasks that we might want to do such as:
- Reading/writing raster files.
- Visualizing the raster data.
- Processing the data with HPC capabilities (`NumPy` + `Dask`).
- Geospatial operations such as...
    - Resampling.
    - Clipping.
    - Reprojecting.

In [ ]:
import rioxarray as rxr

# Read the raster Tiff file
ds = rxr.open_rasterio(data_dir / "elevation.tiff")
ds

We can see that the data is essentially a multi-dimensional array of values. That has the following components:
- **Dimensions**: `band` (only elevation), `y` (latitude), `x` (longitude).
- **Coordinates**: specify the dimension tick values on the multi-dimensional array.
- **Attributes**: that come with the data.

In [ ]:
# Visualize the array
_ = ds.plot(robust=True, cmap="terrain")

After this quick introduction to geospatial data, let's move to learning more about the field by trying to solve a problem.

---

# Case study: *Crop Type Classification in Africa*

In this project, we will use Geospatial machine leraning to classify farm-level crop types in Kenya using Sentinel-2 satellite imagery.

<div style="text-align:center;">
    <figure>
        <img style="width:50%;" src="https://zindi-public-release.s3.eu-west-2.amazonaws.com/uploads/competition/image/42/header_e7a684a3-b7c0-4f53-81ca-7406f148fc5e.png" />
        <figcaption style="font-size:small;">Question: <b>What is the farmed crop for each field?</b> (Image credit: <a href="https://zindi.africa/">Zindi</a>)</figcaption>
    </figure>
</div>

## Problem Scoping

Let's answer a few fundamental questions about the problem before we begin:

#### **What type of machine learning problem is this?**

This is a supervised multiclass classification problem.

#### **What inputs are we going to use?**

We will use pixel-level Sentinel-2 satellite imagery as input to our model:

<div style="text-align:center;">
    <figure>
        <img style="width:50%;" src="https://images.ctfassets.net/qfhr9fiom9gi/l1YijPOaC0jOT4pIs4FFq/c308480fc0a7ecef82c6724a4113ed7c/pasted_image_0.png" />
        <figcaption style="font-size:small;">Sentinel-2 Bands (reference <a href="https://www.mdpi.com/2072-4292/8/3/166">paper</a>)</figcaption>
    </figure>
</div>

- The input includes **12 bands of observations from Sentinel-2 L2A**: observations in the ultra-blue, blue, green, red; visible and near-infrared (VNIR); and short wave infrared (SWIR) spectra, as well as a cloud probability layer.
- Each pixel has measurements for **13 dates** that cover the whole farming season.

Details about the bands:
- The twelve bands are `[B01, B02, B03, B04, B05, B06, B07, B08, B8A, B09, B11, B12]`.
    - B01 (Coastal aerosol)
    - B02 (Blue)
    - B03 (Green)
    - B04 (Red)
    - B05 (Red Edge 1)
    - B06 (Red Edge 2)
    - B07 (Red Edge 3)
    - B08 (NIR - Near Infrared)
    - B8A (Red Edge 4)
    - B09 (Water vapor)
    - B11 (SWIR - Shortwave Infrared 1)
    - B12 (SWIR - Shortwave Infrared 2)
- The cloud probability layer is a product of the Sentinel-2 atmospheric correction algorithm (Sen2Cor) and provides an estimated cloud probability (0-100%) per pixel.

#### **What are we predicting?**

We need to **classify each farm** into one of the following categories:

```
Crop ID   Crop Type
   1      Maize
   2      Cassava
   3      Common Bean
   4      Maize & Common Bean (intercropping)
   5      Maize & Cassava (intercropping)
   6      Maize & Soybean (intercropping)
   7      Cassava & Common Bean (intercropping)
```

#### **How will we validate the model?**

We will conduct a random train-validation split by farm IDs.

#### **How will we measure performance?**

The evaluation metric is **cross-entropy**. For each farm `field ID`, we are expected to predict the probability that the farm has a crop of that type. Example:

#### Prediction
```
FieldID     CropId_1  CropId_2  CropId_3  CropId_4  CropId_5  CropId_6  CropId_7
<integer>   <float>   <float>   <float>   <float>   <float>   <float>   <float>
  1184       0.14       0.14      0.14      0.14      0.14      0.14      0.16
```

#### Target
```
FieldID     CropId_1  CropId_2  CropId_3  CropId_4  CropId_5  CropId_6  CropId_7
<integer>   <float>   <float>   <float>   <float>   <float>   <float>   <float>
  1184         0         0         1         0         0         0         0
```

---

Next, we want to prepare the dataset for machine learning.

## Data Preparation

In this section, we want to do the following:

1. Remove the pixels where the cloud probability value is greater than `50%`
2. Split the data into train/validation/test.
3. Verify that no data leakage is present in the train/validation/test data.
4. Check the distribution of each channel or band.
5. Plot the farms by their labels in a map.
6. Visualize a single farm's NDVI as it changes through time (13 dates).

In [ ]:
import pandas as pd

df = pd.read_feather(data_dir / "df.feather")
df.head()

Each `(pixel, time)` is a row. Let's start by removing the pixels that are cloudy:

In [ ]:
# Drop pixels that have a cloud cover greater than 50
df = df[df["CLD"] < 50]

# No need to keep the `CLD` column anymore
df = df.drop(columns=["CLD"])

Let's quickly check if we have missing values:

In [ ]:
import numpy as np

np.any(df.isna())

Let's conduct the train/validation/test split:

In [ ]:
# Set the seed for reproducibility
import numpy as np
np.random.seed(42)

# NOTE: the `deploy` labels are hidden and marked with `crop == 0`
# We use the label `crop == 0` to get the `deploy` frame and ignore it
deploy_crop_id = 0
deploy = df[df["crop"] == deploy_crop_id]

# Train/Validation/Test are the remaining rows
train_val_test = df[~df["field"].isin(deploy["field"])]

# Get the unique field IDs from the train/validation/Test rows
train_val_test_field_ids = train_val_test["field"].sample(frac=1).unique()

# Randomly select 80/10/10 split for train/val/test
val_field_ids = np.random.choice(train_val_test_field_ids, size=int(len(train_val_test_field_ids) * 0.1), replace=False)
test_field_ids = np.random.choice(list(set(train_val_test_field_ids) - set(val_field_ids)), size=int(len(train_val_test_field_ids) * 0.1), replace=False)
train_field_ids = list(set(train_val_test_field_ids) - set(val_field_ids) - set(test_field_ids))

# Create `train`, `val`, and `test` sets based on the validation field IDs
train = train_val_test[train_val_test["field"].isin(train_field_ids)]
val = train_val_test[train_val_test["field"].isin(val_field_ids)]
test = train_val_test[train_val_test["field"].isin(test_field_ids)]

# print the shapes of the train/val/test sets
train.shape, val.shape, test.shape

Let's verify that no data leakage is happening. We define leakage as follows:

> A validation or test farm pixels in the training dataframe (or the reverse).

In [ ]:
# Verify that the sets of field IDs from `train`, `val`, and `test` are mutually exclusive
assert len(set(train["field"].unique()).intersection(set(val["field"].unique()))) == 0
assert len(set(train["field"].unique()).intersection(set(test["field"].unique()))) == 0
assert len(set(val["field"].unique()).intersection(set(test["field"].unique()))) == 0

Next, let's check the distribution of the band values we have:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

g = sns.displot(data=train.drop(columns=["time", "lat", "lon", "field", "crop"]).melt().sample(100_000), x='value', hue="band", multiple='stack')
plt.title('Distribution of Input for each Band')
plt.show()

Let's visualize the spatial distribution of the farms by their crop IDs:

In [ ]:
import contextily as ctx
from shapely.geometry import Polygon

# Create a new GeoDataFrame
d = train[["field", "lon", "lat", "crop"]].copy()

# Map crop IDs to names
id_to_name = {
    1: 'Maize',
    2: 'Cassava',
    3: 'Common Bean',
    4: 'Maize & Common Bean (intercropping)',
    5: 'Maize & Cassava (intercropping)',
    6: 'Maize & Soybean (intercropping)',
    7: 'Cassava & Common Bean (intercropping)',
}

# Replace the 'crop' column with mapped names
d['crop'] = d['crop'].map(id_to_name)

# Group by field and crop, and create polygons from point coordinates
polygons = d.groupby(['field', 'crop']).apply(lambda df: Polygon(zip(df.lon, df.lat))).reset_index()
polygons.columns = ['field', 'crop', 'geometry']

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(polygons, geometry='geometry', crs="EPSG:4326")

# Create a figure and an axes object
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the GeoDataFrame using the 'crop' column to color the polygons
gdf.plot(column="crop", legend=True, ax=ax, cmap="Accent")

# TODO: comment to check the spatial coverage of all training labels
ax.set_xlim([34.2, 34.3])
ax.set_ylim([.5, .6])

# Add a basemap
ctx.add_basemap(ax, crs=gdf.crs.to_string(), source=ctx.providers.Stamen.Terrain)

# Show the plot
plt.show()

Every field has a history of 13 dates across the growing farming season. We one field's NDVI evolution.

To emphasize vegetation, a common technique in remote sensing is to use the Normalized Difference Vegetation Index (NDVI). NDVI is a measure of the amount and condition of green vegetation present. The NDVI is calculated from the visible and near-infrared light reflected by vegetation. The formula for NDVI is:

$$NDVI = \frac{NIR-Red}{NIR+Red}$$

In [ ]:
from random import choice

d = train.copy()
field_id = choice(d["field"].unique().tolist())
d = d.loc[d["field"] == field_id]
d = d[["time", "lat", "lon", "B08", "B04"]]
d = d.melt(id_vars=["time", "lat", "lon"], value_vars=["B08", "B04"], var_name="band", value_name="value")
d = d.set_index(["time", "lat", "lon", "band"])
d = d.to_xarray()

# Calculate NDVI and assign to 'value'
d = (d.sel(band='B08') - d.sel(band='B04')) / (d.sel(band='B08') + d.sel(band='B04'))

# Normalize NDVI to the range 0-255
d = (d.apply(lambda x: (x - x.min()) / (x.max() - x.min())) * 255).astype("uint8")

# Plot the field (each column should represent a time)
_ = d["value"].plot.imshow(col="time", x="lon", y="lat", col_wrap=5, figsize=(13, 7))

There are many more things that we can explore with data. For now, let's skip ahead to the **modeling** section.

---

## Modeling

In this section, we aim to train a `LightGBM` model to predict each farm's crop type by summarizing the historical band information. We will go over the following:

<div style="text-align:center;">
    <figure>
        <img style="width:66%;" src="https://i.imgur.com/Brd7Jqe.png" />
        <figcaption style="font-size:small;">Processing steps</figcaption>
    </figure>
</div>

- Establishing the validation metric of a **frequency based model** that always predicts crop type frequencies derived from `y_train`.
- Feature engineering: we will calculate the following S2-based indidces:
$$
\begin{align*}
\text{NDVI} & = \frac{{B08 - B04}}{{B08 + B04}} \\
\text{RDNDVI1} & = \frac{{B08 - B05}}{{B08 + B05}} \\
\text{RDNDVI2} & = \frac{{B08 - B06}}{{B08 + B06}} \\
\text{GCVI} & = \frac{{B08}}{{B03}} - 1 \\
\text{RDGCVI1} & = \frac{{B08}}{{B05}} - 1 \\
\text{RDGCVI2} & = \frac{{B08}}{{B06}} - 1 \\
\text{MTCI} & = \frac{{B08 - B05}}{{B05 - B04}} \\
\text{MTCI2} & = \frac{{B06 - B05}}{{B05 - B04}} \\
\text{REIP} & = 700 + 40 \left( \frac{{(B04 + B07)/2 - B05}}{{B07 - B05}} \right) \\
\text{NBR1} & = \frac{{B08 - B11}}{{B08 + B11}} \\
\text{NBR2} & = \frac{{B08 - B12}}{{B08 + B12}} \\
\text{NDTI} & = \frac{{B11 - B12}}{{B11 + B12}} \\
\text{CRC} & = \frac{{B11 - B03}}{{B11 + B03}} \\
\text{STI} & = \frac{{B11}}{{B12}}
\end{align*}
$$
- **Spatial median-aggregation** by field `ID` and `time`.
- Conduct **period-based temporal aggregation** and for each band and index, create period-based columns using the following temporal groups:
    - `period 1`
        - *2019-06-06*
    - `period 2`
        - *2019-07-01*
        - *2019-07-06*
        - *2019-07-11*
        - *2019-07-21*
    - `period 3`
        - *2019-08-05*
        - *2019-08-15*
        - *2019-08-25*
    - `period 4`
        - *2019-09-09*
        - *2019-09-19*
        - *2019-09-24*
        - *2019-10-04*
    - `period 5`
        - *2019-11-03*

### Frequency baseline

A simple algorithm can:

1. Calculate the frequency of each crop type from the training data.
2. Predicts the same frequencies for each validation field.

The resulting metric serves to filter out any subsequent models.

In [ ]:
def prepare_Xy(df):
    d = df.copy()
    d = d.groupby(["field", "time"], as_index=False).mean()
    d = d.drop("time", axis=1).groupby("field", as_index=False).mean()
    return d.drop(['field', 'crop'], axis=1), d['crop']

In [ ]:
X_train, y_train = prepare_Xy(train)
X_val, y_val = prepare_Xy(val)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [ ]:
# Calculate the class frequencies from `y_train` in order to generate the baseline predictions
y_val_hat = np.repeat(y_train.value_counts(normalize=True).sort_index().values[None,...], y_val.shape[0], axis=0)
y_val_hat.shape

In [ ]:
from sklearn.metrics import log_loss

# Calculate cross-entropy
cross_entropy = log_loss(y_val, y_val_hat)
print(f'Cross-entropy is {cross_entropy}')

In [ ]:
# .. to be used for comparison
baseline_ce = cross_entropy
baseline_ce

Any model that we construct should have a validation cross-entropy less than the baseline cross-entropy.

### `LightGBM`

We will create functions that cover the data preparation steps in the original section description.

Let's implement the feature engineering function that would add additional vegetation indices of interest:

In [ ]:
def calculate_indices(df):
    """
    Compute various spectral indices commonly used in remote sensing for vegetation monitoring.

    Parameters
    ----------
    df : pandas.DataFrame
        Input DataFrame containing columns for the different band values.

    Returns
    -------
    pandas.DataFrame
        The DataFrame with added columns for the calculated indices.
    """
    # Make a copy of the dataframe to avoid SettingWithCopyWarning
    df = df.copy()

    # Normalized Difference Vegetation Index (NDVI)
    df['NDVI'] = (df['B08'] - df['B04']) / (df['B08'] + df['B04'])

    # Red-edge Normalized Difference Vegetation Index (RDNDVI)
    df['RDNDVI1'] = (df['B08'] - df['B05']) / (df['B08'] + df['B05'])
    df['RDNDVI2'] = (df['B08'] - df['B06']) / (df['B08'] + df['B06'])

    # Green Chlorophyll Vegetation Index (GCVI)
    df['GCVI'] = df['B08'] / df['B03'] - 1

    # Red-edge GCVI
    df['RDGCVI1'] = df['B08'] / df['B05'] - 1
    df['RDGCVI2'] = df['B08'] / df['B06'] - 1

    # Meris Terrestrial Chlorophyll Index (MTCI)
    df['MTCI'] = (df['B08'] - df['B05']) / (df['B05'] - df['B04'])
    df['MTCI2'] = (df['B06'] - df['B05']) / (df['B05'] - df['B04'])

    # Red-edge Inflection Point (REIP)
    df['REIP'] = 700 + 40 * (((df['B04'] + df['B07']) / 2) - df['B05']) / (df['B07'] - df['B05'])

    # Normalized Burn Ratio (NBR)
    df['NBR1'] = (df['B08'] - df['B11']) / (df['B08'] + df['B11'])
    df['NBR2'] = (df['B08'] - df['B12']) / (df['B08'] + df['B12'])

    # Normalized Difference Tillage Index (NDTI)
    df['NDTI'] = (df['B11'] - df['B12']) / (df['B11'] + df['B12'])

    # Canopy Chlorophyll Content Index (CRC)
    df['CRC'] = (df['B11'] - df['B03']) / (df['B11'] + df['B03'])

    # Soil Tillage Index (STI)
    df['STI'] = df['B11'] / df['B12']

    return df

We also need function for spatial and temporal aggregation to reduce the dimensionality of the dataset:

In [ ]:
def spatial_median_aggregation(df, bands):
    """
    Aggregate data by field and time, using the median of band values.

    Parameters
    ----------
    df : pandas.DataFrame
        Input DataFrame with 'field', 'time', and band columns.
    bands : list
        List of band columns to be aggregated.

    Returns
    -------
    pandas.DataFrame
        Aggregated DataFrame with median band values.
    """
    # Calculate median of band values for each unique 'field' and 'time'
    agg_df = df.groupby(['field', 'time'])[bands].median().reset_index()

    # Drop duplicate entries for each unique 'field' and 'time', and remove band columns
    unique_df = df.drop_duplicates(['field', 'time']).drop(bands, axis=1)

    # Merge aggregated DataFrame with unique DataFrame
    return pd.merge(agg_df, unique_df, on=['field', 'time'])


def period_based_aggregation(df, bands):
    """
    Aggregate data by field and defined time periods, using the mean of band values.

    Parameters
    ----------
    df : pandas.DataFrame
        Input DataFrame with 'field', 'time', and band columns.
    bands : list
        List of band columns to be aggregated.

    Returns
    -------
    pandas.DataFrame
        Aggregated DataFrame with mean band values for each time period.
    """
    # Define time periods
    periods = {
        'p1': pd.to_datetime(['2019-06-06']),
        'p2': pd.to_datetime(['2019-07-01', '2019-07-06', '2019-07-11', '2019-07-21']),
        'p3': pd.to_datetime(['2019-08-05', '2019-08-15', '2019-08-25']),
        'p4': pd.to_datetime(['2019-09-09', '2019-09-19', '2019-09-24', '2019-10-04']),
        'p5': pd.to_datetime(['2019-11-03'])
    }

    # Assign period labels based on 'time'
    for period, dates in periods.items():
        df.loc[df['time'].isin(dates), 'period'] = period

    # Calculate mean of band values for each unique 'field' and 'period'
    period_agg_df = df.groupby(['field', 'period'])[bands].mean().reset_index()

    # Drop duplicate entries for each unique 'field' and 'period', and remove band columns
    unique_df = df.drop_duplicates(['field', 'period']).drop(bands, axis=1)

    # Merge aggregated DataFrame with unique DataFrame
    return pd.merge(period_agg_df, unique_df, on=['field', 'period'])

Finally, we create functions to pivot the table (making periods into columns) and another function that runs the steps and splits the dataframe into `X` and `y`:

In [ ]:
def pivot_dataframe(df):
    """
    Pivot the DataFrame so that each time period becomes a separate column.

    Parameters
    ----------
    df : pandas.DataFrame
        Input DataFrame with 'field', 'period', and other columns.

    Returns
    -------
    pandas.DataFrame
        Pivoted DataFrame with each 'period' as a separate column.
    """
    return df.pivot(index=['field', 'crop', 'lat', 'lon'], columns='period').fillna(-1).reset_index()


def process_dataframe(df, bands):
    """
    Process the DataFrame by calculating indices, aggregating data, and pivoting.

    Parameters
    ----------
    df : pandas.DataFrame
        Input DataFrame with 'field', 'time', and band columns.
    bands : list
        List of band columns to be processed.

    Returns
    -------
    X : pandas.DataFrame
        Processed DataFrame with features for machine learning.
    y : pandas.Series
        Target labels for machine learning.
    """
    # Calculate spectral indices
    df = calculate_indices(df)

    # Aggregate data by field and time using spatial median
    df = spatial_median_aggregation(df, bands)

    # Aggregate data by field and time period using mean
    df = period_based_aggregation(df, bands)

    # Calculate average latitude and longitude for each field
    lat_lon_agg = df.groupby('field')[['lat', 'lon']].mean().reset_index()

    # Merge aggregated DataFrame with latitude and longitude DataFrame
    df = pd.merge(df.drop(columns=['lat', 'lon']), lat_lon_agg, on='field', how='left')

    # Pivot DataFrame to have each period as a separate column
    df = pivot_dataframe(df)

    # Flatten multi-level column names
    df.columns = [''.join(col).strip() if isinstance(col, tuple) else col for col in df.columns.values]

    # Select columns to keep
    columns_to_keep = ['field', 'lat', 'lon', 'crop'] + [col for col in df.columns if col.endswith(('p1', 'p2', 'p3', 'p4', 'p5')) and not col.startswith(('time', 'lat', 'lon', 'crop'))]
    df = df[columns_to_keep]

    # Split DataFrame into features (X) and target labels (y)
    X, y = df.drop(["crop"], axis=1), df["crop"]

    return X, y

Let's prepare the training, validation, and test arrays:

In [ ]:
# Set the band columns'
bands = ['B04', 'B05', 'B06', 'B07', 'B08', 'B11', 'B12', 'NDVI', 'RDNDVI1', 'RDNDVI2', 'GCVI', 'RDGCVI1', 'RDGCVI2', 'MTCI', 'MTCI2', 'REIP', 'NBR1', 'NBR2', 'NDTI', 'CRC', 'STI']

# Prepare the dataset
print("Processing `train` ...")
X_train, y_train = process_dataframe(train, bands)

print("Processing `val` ...")
X_val, y_val = process_dataframe(val, bands)

print("Processing `test` ...")
X_test, y_test = process_dataframe(test, bands)

# Print the shapes
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

Now, let's conduct random hyperparameter search with cross-validation using the `LightGBM` estimator:

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
import lightgbm as lgb

# Define the LightGBM model
model = lgb.LGBMClassifier(objective="multiclass", verbose=-1, num_class=7)
banned_cols = ["field"]

# Define the hyperparameters space
param_dist = {
    'num_leaves': [31, 127, 200, 300],
    'reg_alpha': [0.1, 0.5],
    'min_data_in_leaf': [30, 50, 100, 300, 400],
    'lambda_l1': [0, 1, 1.5],
    'lambda_l2': [0, 1]
}

# Define the scorer
scorer = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

# Randomized Search for hyperparameter tuning
random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=15, scoring=scorer, cv=3, verbose=1, n_jobs=-1)
random_search.fit(X_train.drop(banned_cols, axis=1), y_train)

## Evaluation

We re-train the best estimator on the training data and get the validation cross-entropy:

In [ ]:
# Create the LightGBM model instance with the best hyperparameters
model = lgb.LGBMClassifier(objective="multiclass", num_class=7, verbose=-1, **random_search.best_params_)

# Fit the model to the training set
model.fit(X_train.drop(banned_cols, axis=1), y_train)

# Predict the validation set results
y_val_hat = model.predict_proba(X_val.drop(banned_cols, axis=1))

# Report cross-entropy
print(f"Cross-entropy with best hyperparameters is {log_loss(y_val, y_val_hat):.5f}")
print(f"It is {100*(log_loss(y_val, y_val_hat) - baseline_ce)/baseline_ce:.2f}% better than the baseline")

### Investigating Class-Imbalances

Let's report the following metrics on the combination of validation + test points:
- `Precision`
- `Recall`
- `F1`
- `Confusion matrix`

In [ ]:
# Predict the validation set results
y_test_hat = model.predict(pd.concat([X_val, X_test]).drop(banned_cols, axis=1))
y_test_arr = pd.concat([y_val, y_test]).values
y_test_hat.shape, y_test_arr.shape

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate precision, recall, and F1 score
precision = precision_score(y_test_arr, y_test_hat, average='weighted')
recall = recall_score(y_test_arr, y_test_hat, average='weighted')
f1 = f1_score(y_test_arr, y_test_hat, average='weighted')

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

In [ ]:
# Calculate confusion matrix
cm = confusion_matrix(y_test_arr, y_test_hat, normalize="true")

# Plot confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, xticklabels=id_to_name.values(), yticklabels=id_to_name.values())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

Except for `maize` (which is majority class), we are not doing well classifiying the other minority crop classes. Since cross-entropy does not mitigate against class imbalance, we still get a good score.

*Hint: try changing `LGBMClassifier`'s `class_weight` attribute to `balanced`.*

### XAI

What are the most important periods and indices?

In [ ]:
import shap

# Prepare the validation + test data for the model
X_vt = pd.concat([X_val, X_test])

# explain the model's predictions using SHAP
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_vt.drop(banned_cols, axis=1))

shap.summary_plot(shap_values, X_vt.drop(banned_cols, axis=1))

Let's figure out which periods are the most important:

In [ ]:
# Compute the absolute SHAP values for each feature
abs_shap_values = np.sum(np.abs(shap_values), axis=(0, 1))

# Get the feature names
feature_names = X_vt.drop(banned_cols, axis=1).columns

# Create a DataFrame linking feature names to their importance
feature_importances = pd.DataFrame({
    'feature': feature_names,
    'importance': abs_shap_values
})

# Sort the DataFrame by importance in descending order
feature_importances = feature_importances.sort_values('importance', ascending=False)

# Drop `lat` and `lon` from the dataset
feature_importances = feature_importances[feature_importances["feature"].isin(["lat", "lon"]) == False]

# Normalize the feature importances to sum to one
feature_importances['importance'] = feature_importances['importance'] / feature_importances['importance'].sum()

# Split the feature name into `index` and `period` (period is the last two characters)
feature_importances['period'] = feature_importances['feature'].str[-2:]
feature_importances['index'] = feature_importances['feature'].str[:-2]
feature_importances = feature_importances.drop("feature", axis=1)

# Get the most important periods separately by aggregating their importance
periods = feature_importances.drop("index", axis=1).groupby('period').sum().sort_values('importance', ascending=False)

# Get the most important bands separately by aggregating their importance
bands = feature_importances.drop("period", axis=1).groupby('index').sum().sort_values('importance', ascending=False)

In [ ]:
periods

In [ ]:
bands.iloc[:10]

We highlight the following top indices:

- `NBR2`: Normalized Burn Ratio 2. It is used in remote sensing to identify burned areas. It uses the Near Infrared (NIR) and Shortwave Infrared (SWIR) portions of the electromagnetic spectrum.
- `CRC`: Canopy Chlorophyll Content Index. It is used to estimate the chlorophyll content in plant canopies.
- `B01`: Band 1 of Sentinel-2. This is a coastal aerosol band and captures light in the blue portion of the electromagnetic spectrum.
- `B08`: Band 8 of Sentinel-2. This is a NIR (Near Infrared) band. It is often used in vegetation analysis as it reflects maximum light in healthy vegetation.
- `GCVI`: Green Chlorophyll Vegetation Index. It is used to measure the chlorophyll content of vegetation by using the Green and Near-Infrared bands.
- `REIP`: Red Edge Inflection Point. It is used in vegetation studies to indicate the boundary between the red and NIR region of the spectrum where vegetation has a strong reflection.
- `B8A`: Band 8a of Sentinel-2. This is a Narrow NIR (Near Infrared) band. It is used to study water bodies and vegetation.
- `B06`: Band 6 of Sentinel-2. It is a red-edge band, which is useful for vegetation studies.
- `RDNDVI2`: Ratio Divergence Normalized Difference Vegetation Index 2. This is presumably a custom vegetation index that uses a ratio and divergence calculation to normalize the vegetation index.
- `B09`: Band 9 of Sentinel-2. It is a water vapor band, useful for atmospheric studies.

---

## Inference

In this section, we will report the final metrics on the validation set and visualize the farms with their crop types:

In [ ]:
# Predict on the test set
y_test_pred = model.predict_proba(X_test.drop(banned_cols, axis=1))
y_test_pred.shape

In [ ]:
# Export the results
report = X_test[["field"]].copy()

# Create the Crop_ID_1,Crop_ID_2,Crop_ID_3,Crop_ID_4,Crop_ID_5,Crop_ID_6,Crop_ID_7 columns and assign the predictions
cols = ['Crop_ID_1','Crop_ID_2','Crop_ID_3','Crop_ID_4','Crop_ID_5','Crop_ID_6','Crop_ID_7']
report[cols] = y_test_pred
report

Let's visualize the predicted test farms:

In [ ]:
from shapely.geometry import Point, LineString, Polygon

def create_geometry(df):
    coords = list(zip(df.lon, df.lat))
    if len(coords) == 1: return Point(coords[0])
    elif len(coords) == 2: return LineString(coords)
    else: return Polygon(coords)

# Create the polygons from the test set
d = test.copy()
cols = ["field", "lat", "lon"]
d = d[cols].drop_duplicates()
d = d.groupby('field').apply(create_geometry).reset_index().rename(columns={0: "geometry"})

# Create the dataframe to hold the pixel locations and the predicted crop types
report = X_test.copy()
report = report[["field"]]
report["crop"] = y_test_pred.argmax(axis=1) + 1

# Merge the two dataframes
report = report.merge(d, on="field", how="left").rename(columns={0: "geometry"})
report = gpd.GeoDataFrame(report, geometry="geometry")

# Replace the 'crop' column with mapped names
report['crop'] = report['crop'].map(id_to_name)

# Plot
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the GeoDataFrame using the 'crop' column to color the polygons
report.plot(column="crop", legend=True, ax=ax, cmap="Accent")

# Add a basemap
ctx.add_basemap(ax, crs="EPSG:4326", source=ctx.providers.Stamen.Terrain)

# Show the plot
plt.show()

---

# Conclusion

In this notebook, we explored the broad and exciting field of geospatial machine learning. We began with a brief introduction to geospatial data, both in terms of vector and raster information. We then defined our problem and proceeded to gather and preprocess the necessary data to address it.

After preparing our data, we fitted a machine learning model and evaluated its performance against the human baseline. We then used our model to make predictions, allowing us to see the power and potential of applying machine learning techniques to geospatial data.

Finally, we expanded our horizons by discussing the use of more advanced deep learning models for solving similar problems. This exploration underlines the fact that the techniques and approaches we've covered here are just the beginning. There's a wealth of more complex and powerful tools available in the geospatial machine learning space, and we encourage you to continue exploring!

Remember, the journey of learning never ends, and each step brings us closer to making meaningful contributions to the field. Happy learning!

---

# Resources

### Tutorials

- [Geospatial Primer](https://github.com/Akramz/geospatial-primer).
- [Introduction to Geospatial Data](https://colab.research.google.com/drive/1-85h5tEB0AJYT8xQ5H1wtSnCafXuLTHo#scrollTo=JDT5jUmCiTH-).
- [Geospatial Data Analysis](https://colab.research.google.com/drive/1Yfkm63OV3eCtR3IVB-4owi2DJgj2Wd84).
- [Geospatial Deep learning: Getting started with TorchGeo](https://pytorch.org/blog/geospatial-deep-learning-with-torchgeo/).
- [Automating GIS-processes Course]((https://autogis-site.readthedocs.io/en/latest/))
- [Geospatial Data with Python: Shapely and Fiona](https://macwright.com/2012/10/31/gis-with-python-shapely-fiona.html)
- [Introduction to Raster Data Processing in Open Source Python](https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/raster-data-processing/).
- [XArray fundamental](https://rabernat.github.io/research_computing_2018/xarray.html).
- [XArray tutorials](https://github.com/xarray-contrib/xarray-tutorial).
- [Visualization: contextily tutorial](https://geopandas.org/en/stable/gallery/plotting_basemap_background.html).


### Geospatial Libraries

- [Shapely](https://github.com/shapely/shapely).
- [GeoPandas](https://github.com/geopandas/geopandas).
- [Contextily](https://github.com/geopandas/contextily).
- [Rasterio](https://github.com/rasterio/rasterio).
- [Xarray](https://github.com/pydata/xarray).
- [RioXarray](https://github.com/corteva/rioxarray).
- [TorchGeo](https://github.com/microsoft/torchgeo).

### Credits

- [CV4A Kenya Crop Type Competition (source dataset)](https://mlhub.earth/data/ref_african_crops_kenya_02).
- [Related Zindi Competition](https://zindi.africa/competitions/iclr-workshop-challenge-2-radiant-earth-computer-vision-for-crop-recognition/).
- Paper: Jin, Zhenong, et al. "[Smallholder maize area and yield mapping at national scales with Google Earth Engine.](https://web.stanford.edu/~mburke/papers/jin%20et%20al%202019.pdf)" Remote Sensing of Environment 228 (2019): 115-128.
- [Gilles Q. HACHEME](https://www.linkedin.com/in/gilles-q-hacheme-a0956ab7/).
- [Aisha Alaagib](https://sa.linkedin.com/in/aishaalaagib).
- [Akram Zaytar](https://www.microsoft.com/en-us/research/people/akramzaytar/).
- [Girmaw Abebe Tadesse](https://www.microsoft.com/en-us/research/people/gtadesse/).

---